# Cross Referencing Overview

![a](https://img.shields.io/badge/Subject:-Overview-blue)
![b](https://img.shields.io/badge/Difficulty:-Easy-green)
![c](https://img.shields.io/badge/Author:-Eliza_Diggins-green)

---

This example is intended to give a "bird's eye" overview of ``pyXMIP``, what it can do, how to perform the basic analyses, etc!

Generally speaking, ``pyXMIP`` is designed with the following steps in mind:

1. **Load / explore a source catalog**
   - This is where you provide the data that needs to be cross-identified. There are a variety of things ``pyXMIP`` can do, but almost all of them require you to provide a source catalog as a first step.
2. **Cross-match**
   - The first step in cross-identification (determining a known source as a match to a given detection) is a process which we refer to as "cross-matching". In the cross-matching step, ``pyXMIP`` explores a set of external data sources and finds a list (or lists) of *potential* candidates for the cross-identification of each source in your catalog.
3. **Reduction**
   - In the 3rd step, we go from the cross-matching phase (many possible matches to each source) to the cross-identification phase (one match to a given source). In this part of the process, the user gets to decide how ``pyXMIP`` should evaluate the "best" match to a given source.
4. **Science Outcomes**
   - The software has done its work, this part is up to you! We provide some tools for performing match quality cutoffs, plotting various aspects of the cross-matching catalog, etc.

In this guide, we're going to go through all of these steps for a small example case to showcase the basic ideas!

---

## Contents

- [Loading Data](#Loading-Data)
  - [Schemas](#Schemas)
- [Cross Matching](#Cross-Matching)
  - [Performing the Cross-Match](#Cross-Matching-Against-Databases)
  - [Cross Match Databases](#Cross-Match-Databases)
- [Reduction](#Reduction)

---



## Loading Data

The first step is to generate a ``SourceTable`` containing your catalog of novel sources. 

> A ``SourceTable`` is just a fancy version of the typical ``astropy`` ``Table`` class. You can treat it more-or-less like a ``pandas`` or ``astropy``
> table. In addition to basic functionality, these classes also have some specific functions to aid the user (as you'll see below).



In this example, we're going to use the **eROSITA X-ray Survey (eRASS1)**!
Let's go ahead and get started by downloading the hard band X-ray catalog [here](https://erosita.mpe.mpg.de/dr1/AllSkySurveyData_dr1/Catalogues_dr1/MerloniA_DR1/eRASS1_Hard.tar.gz).
Once it's been extracted, there should be a file ``eRASS1_Hard.v1.0.fits``.


In [1]:
import pyXMIP as pyxmip

# Load the catalog table just like a normal astropy table.
catalog = pyxmip.SourceTable.read("data/eRASS1_Hard.v1.0.fits")

Just like any other table, you can look at the catalog data by indexing:

In [2]:
catalog[:2]

### Schemas

What makes ``SourceTable`` instances different from the traditional ``astropy.Table`` class is that every ``SourceTable`` comes with a ``SourceTableSchema`` instance.

> ``Schema`` classes appear a lot in ``pyXMIP``. Effectively, ``Schema`` classes are how ``pyXMIP`` figures out what different parts of user-provided
> data mean. For ``SourceTable`` instances, this is largely about translating the column names into standardized names that ``pyXMIP`` understands.

In many cases, the ``Schema`` may be automatically generated simply by accessing them:

In [3]:
catschema = catalog.schema

> What's with all the ``DEBUG`` statements?
>
> Effectively, ``pyXMIP`` is going through the columns of our catalog and looking for recognizable names. Here, it found ``RA`` and ``DEC`` as well as
> ``LII`` and ``BII`` (galactic coordinates). It also found a source name column (``IAUNAME``). Many of the other columns it looked for (like the
> redshift) weren't found. Finally, it also determined that there were 2 possible coordinate systems and selected a default (ICRS).

Because ``SourceCatalog`` instances have ``Schema``, we can use general attributes to access data regardless of the column name:

In [4]:
import numpy as np
from astropy.units import Quantity

print(f"There are {len(catalog)} sources in the catalog.")
_low_gal_lat = catalog[np.abs(Quantity(catalog.GAL_B).to_value("deg")) < 1]
print(f"There are {len(_low_gal_lat)} sources within the galactic plane.")

In some cases, the automatically generated schema is a bit insufficient. For example, in this case, there are RA / DEC error column that weren't picked up by the naive search. We can add these in manually!

In [33]:
catalog.schema.column_map.RA_ERR = {"name": "RA_UPERR", "unit": "arcsec"}
catalog.schema.column_map.DEC_ERR = {"name": "DEC_UPERR", "unit": "arcsec"}

# Fetch the errors
import matplotlib.pyplot as plt

plt.hist(catalog.RA_ERR, ec="k", fc="darkgreen", bins=np.geomspace(1e-3, 40))
plt.xscale("log")
plt.yscale("log")
plt.ylabel("Number of eRASS1 Sources")
plt.xlabel(r"$\sup \sigma_{\mathrm{RA}}$, $[\mathrm{arcsec}]$")

We can also use the ``Schema`` to help plot the sources in the catalog!

In [34]:
import matplotlib.pyplot as plt
import numpy as np

# -- create the figure -- #
figure = plt.figure(figsize=(10, 7))
ax = figure.add_subplot(111, projection="aitoff")

# -- pull the latitute longitude, and rate -- #
lat, lon = catalog.lat.to_value("rad"), catalog.lon.to_value("rad")
rate = catalog["ML_RATE_0"]

# -- plot -- #
ax.scatter(lon - np.pi, lat, 1, c=np.log10(rate), cmap="gnuplot_r")

ax.set_ylabel("DEC")
ax.set_xlabel("RA")

We can also visualize in galactic coordinates:

In [35]:
import matplotlib.pyplot as plt
import numpy as np

# -- create the figure -- #
figure = plt.figure(figsize=(10, 7))
ax = figure.add_subplot(111, projection="aitoff")

# -- pull the latitute longitude, and rate -- #
coordinates = catalog.get_coordinates().transform_to("galactic")

lat, lon = coordinates.frame.spherical.lat.rad, coordinates.frame.spherical.lon.rad

rate = catalog["ML_RATE_0"]

# -- plot -- #
ax.scatter(lon - np.pi, lat, 1, c=np.log10(rate), cmap="gnuplot_r")

ax.set_ylabel(r"$b$")
ax.set_xlabel(r"$l$")

> You might be wondering why there are so many sources localized to the blob on the lower right?
>
> This is actually because the observing path of the SRG/eROSITA all-sky survey (eRASS1) leads to overlapping exposure in that region of the sky, thus increasing the source detections.

---

## Cross Matching

We've suceeded in loading the source catalog, the next step is to **identify potential matches**!

``pyXMIP`` is developed with the explicit intention of providing the researcher with the versatility to meet their needs while still providing useful tools. To this end, there are a variety of options for determining candidate matches to the catalog sources. Generically, these fall into two categories:

- **Local Database Searches**: The user provides additional catalogs (typically more ``SourceTables`` or ``.fits`` files) that are then cross-matched against.
  - This is generally a pretty fast process and easily scalable to very large samples.
  - The only real restriction is that sources need to have positions on the sky.
- **Remote Database Searches**: ``pyXMIP`` searches through online databases for potential candidates.
  - Depending on the database selected, these can be slower; however, they are also searching very large repositories of data.
  - ``pyXMIP`` unifies the various APIs for different remote databases to provide a uniform user experience.
  - Additional remote (and local) databases can be easily written as needed by the user.

All of these functionalities are contained in the ``pyXMIP.structures.databases`` module!


---

In order to cross-match our catalog, we have to first find location matches in external databases. All of the available databases are listed in a ``DBRegistry`` (just a list of available databases). Let's see what databases are available to use right now:

In [36]:
from pyXMIP.structures.databases import DEFAULT_DATABASE_REGISTRY

for k, v in DEFAULT_DATABASE_REGISTRY.items():
    print(f"Database {k} corresponds to class {v.__class__.__name__} instance {v.name}")

> **Technical Details**:
>
> Every database is an *instance* of a particular database *class*. The *class* represents the generic database (i.e. SIMBAD or NED), while
> the particular *instance* represents a given set of search / query parameters and conventions (i.e. what columns to return, what format of
>  coordinates to use, etc.)
>
> All of the built-in databases (NED, SIMBAD, etc.) have a ``STD`` instance which simply provides the database with default search settings.

**All** database instances have a ``query_radius`` method, which searches the database for sources within a given region.

In [37]:
from pyXMIP.structures.databases import NED
from astropy.coordinates import SkyCoord
from astropy import units

# Create a default NED instance (default search settings)
ned_instance = NED()

# Search for sources around a position.
# --------- Example ---------- #
# Object: Crab Nebula
# RA: 83.633212, DEC: 22.014460

position = SkyCoord(
    ra=83.633212, dec=22.014460, unit="deg"
)  # The position to search around
search_radius = 2 * units.arcmin  # The search radius

# Perform the query
result = ned_instance.query_radius(position, search_radius)

# Display first 10 results
result[:10]

Just like our ``catalog`` instance, this is another ``SourceTable``! In fact, this one already has a ``SourceTableSchema`` attached to it!

In [38]:
print(result.schema)

> **Technical Details**:
>
> Database classes come with a ``query_schema``, which tells ``pyXMIP`` what schema to attach to the results of a given query. This can be super helpful
> if, for example, you don't want to have to tell ``pyXMIP`` what the different columns mean everytime you query NED or SIMBAD.
>
> If you need to query *your own database*, you might need to provide a ``query_schema`` to tell ``pyXMIP`` what it's looking at!

You may have noticed that the Crab Nebula wasn't in our list of sources! Databases like NED have a lot of different sources in them from a lot of different missions. This means you often get a lot of "catch-all" objects. We'll run into this issue again when we talk about **reduction** (going from cross-matching to cross-identification), but we can already make some headway by sorting for specific types of objects!


In [39]:
# Filter results so that TYPE is SNR (supernova remnant)
filtered_results = result[result.TYPE == "SNR"]

filtered_results

---

### Cross Matching Against Databases

Now that you've been introduced to the ``Database`` classes, it's time to put them to work!

The goal of **cross-matching** is to identify a *large* number of plausible matches to a particular source quickly. From there, ``pyXMIP`` will help you "narrow the field" and determine what the best match is (the so-called **reduction** step).

The ``pyXMIP.cross_reference`` module provides the ``cross_match_table`` function particularly for this purpose:

In [40]:
from pyXMIP.cross_reference import cross_match_table

# Perform the cross-matching process on the first 100 entries of the catalog.
cross_match_table(
    catalog[:100],
    "data/cross_matched.db",
    overwrite=True,
    parallel_kwargs=dict(max_workers=6),
)

> **Wow! That's a lot of output!**
>
> Here's what happened:
> 1. We figured out which databases to cross-match against (in this case, just the default databases).
> 2. We looked for an existing cross-matching output and deleted pre-existing data.
> 3. For each database, we search for each catalog source within a specified radius (configurable). All of the results are then compiled and written to disk.
> 4. The outputs are combined into a single ``SQL`` database with each table representing a different search database.
> 5. A bunch of "post-processing" operations were done to clean up the output and get it ready for use!


The end result of this process is a SQL-database (referred to as a ``CrossMatchDatabase``) at ``data/cross_matched.db``.

### Cross Match Databases

The ``CrossMatchDatabase`` class is the ``pyXMIP``-side representation of the underlying ``SQL`` output from the cross-matching process. It provides a variety of functionality for performing additional analyses on the outputs of your cross-match and (most importantly) provides an interface for the **reduction** step of the cross-identification process.

To load a ``CrossMatchDatabase`` from disk, you need only provide the path to the file:

In [41]:
# load the cross match database from disk
from pyXMIP.cross_reference import CrossMatchDatabase

cmd = CrossMatchDatabase("data/cross_matched.db")

The ``CrossMatchDatabase`` (reflecting the underlying ``SQL``) is composed of "tables":

In [42]:
print(f"The CrossMatchDatabase {cmd} has tables {cmd.tables}")

> **CMD Tables**:
>
> For every ``Database`` class you cross-matched against, you'll see a ``<NAME>_MATCH`` table in your CMD. This contains all of the
>  source candidates from those databases for each of the catalog sources. Additionally, there will always be a ``CATALOG`` table and a ``META`` table.
>
> The ``CATALOG`` table is just a *copy* of your original catalog. This means that your CMD is entirely self-contained and could (in principle)
> be used to create a copy of itself.
>
> The ``META`` table is used for internal book-keeping. Everytime a process or analysis gets run on the CMD, it's added to ``META``. This then
> allows the CMD to avoid rerunning processes and optimizes various procedures.

Just like a normal ``SQL`` database, we can run queries on the ``CrossMatchDatabase``:

In [43]:
# -- Example -- #
# Count the number of proposed matches found in NED for each of the catalog objects.
count_table = cmd.query(
    "SELECT CATOBJ,COUNT('OBJECT NAME') as N FROM NED_STD_MATCH GROUP BY CATOBJ"
)

count_table[:10]  # show the first 10 matches.

> Again, note that most sources have 10+ potential matches. This is why **reduction** is so important for identifying best-candidates.

For some added exploration, let's create a histogram of the number of matches for each source.

In [44]:
import matplotlib.pyplot as plt
import numpy as np

plt.hist(count_table["N"], bins=np.geomspace(1, 1000, 20), ec="k", fc="forestgreen")

plt.xscale("log")
plt.xlabel("Number of matches")
plt.ylabel("Number of sources")

Apparently one of our sources has **several hundred** matches! Let's take a closer look.

In [45]:
# Look up the special source and number of counts.
special_source = count_table.loc[count_table["N"] == np.amax(count_table["N"]), :]
print(special_source)

# Let's get more detail from EROSITA
catalog_entry = cmd.query(
    "SELECT * FROM CATALOG WHERE CATOBJ == '1eRASS J071730.5+374539'"
)

catalog_entry[["LII", "BII", "RA", "DEC", "EXT_LIKE"]]

match_types = cmd.query(
    "SELECT Type, COUNT('OBJECT NAME') FROM NED_STD_MATCH WHERE CATOBJ == '1eRASS J071730.5+374539' GROUP BY Type"
)

In [46]:
match_types

Based on this, we see that there are a number of galaxy clusters (3 - possible referencing the same object) several hundred galaxies and an assortment of other interesting object types.

The question now remains: *How do we go from a list of candidate matches to the optimal match?*


---

## Reduction

It's the **reduction** stage where ``pyXMIP`` really shines! Because (as the scientist), you know more about what you need to do than ``pyXMIP`` ever can, the identification of "best-matches" for each of the catalog objects is a highly configurable procedure.

Starting from the ``CrossMatchDatabase`` (which has many matches to each catalog source), the **reduction** process takes the following general path:

1. **The user determines what criteria should be used to qualify what makes a particular match "good"**

   Some of these criteria are pretty standard (i.e. source types are reasonable given the bandpass, the astrometric error is reasonable, etc). Other criteria may be more specific or the catalog being used comes with additional information that can be used to rule out a potential match. Whatever the case may be, each of these criteria constitutes a ``ReductionProcess``.

   In a technical sense, a ``ReductionProcess`` is a function which acts on a particular table in your ``CrossMatchDatabase`` and spits out a values from $0$ to $1$ for each possible match such that $0$ indicates that the source is highly probably (by that metric) to be a match and $1$ indicates very low likelihood.

2. **The user constructs the relevant reduction process**

   For very common processes, there is likely already a built-in reduction process available. If not, you may need to write your own. Guidance for doing so is provided in our documentation. This can (sometimes) be a tricky task depending on the complexity of the task and (for large databases), attention should be paid to making the process not only correct, but also efficient.

3. **The reductions are run on the ``CrossMatchDatabase``**

   Once you've created the ``ReductionProcess``, it's easy to run the process on your CMD. We'll show how to do this below.

4. **Construct a score for each candidate**

   For each table $T_i$ in your CMD, there will have been a set of $N_i$ reduction processes performed. Each candidate (indexed by $j$) will have a score (from 0 to 1) for the process $T_i$, labeled $\psi_{ij}$.

   Because different criteria may be of *different importance* to the user, each process $T_i$ gets a *weight* $\alpha_i$. The overall score for source $j$ is then

   $$ \xi_j = \frac{1}{Z}\sum_{i} \alpha_i \psi_{ij}, $$

   where

   $$ Z = \sum_i \alpha_i. $$

5. **Combine scores from different tables**

   Each reduction process is run on a table individually (you may use the same process on multiple tables, but you don't *have* to). Thus, you might end up with different scores from different databases. As such, sources from different tables are *combined*.

   > *What about duplicates?*
   >
   > If a potential source appears in two separate tables, then the user may specify a "duplicate-mode" for the scoring (max, min, average, or fixed).
   > If the mode is "fixed", then a particular table always takes precedence over the others in specifying the true score.
   >
   > Once the duplicates have been managed, there will be a single score for each candidate source.

   The rest is easy! Each source candidate has a score $\xi_j \in [0,1]$ and the best match is simply the **minimum** score!

### Astrometry Reduction

---

As an example of a reduction process, we're going to go through the steps of using an astrometric reduction. 

> [**Technical Details**]
>
> The ``AstrometricReductionProcess`` (like all ``ReductionProcess`` classes) takes the table and ``CrossMatchDatabase`` it's operating on as
> arguments. Additionally, the user must specify either (or both) of ``CATALOG_ERR`` or ``DATABASE_ERR``.
>
> ``CATALOG_ERR`` specifies the astrometric precision of the catalog sources.
> ``DATABASE_ERR`` specifies the astrometric precision of the database sources.
>
> Additional resources may be found elsewhere in the documentation with an exhaustive explanation of the details of this procedure.

In [1]:
from pyXMIP.structures.reduction import AstrometricReductionProcess
import pyXMIP as pyxmip
from pyXMIP.cross_reference import CrossMatchDatabase

# Load the catalog table just like a normal astropy table.
catalog = pyxmip.SourceTable.read("data/eRASS1_Hard.v1.0.fits")
cmd = CrossMatchDatabase("data/cross_matched.db")
ARP = AstrometricReductionProcess(
    table="NED_STD_MATCH", cross_match_database=cmd, CATALOG_ERR={}, fill_unknown=True
)

Our ``AstrometricReductionProcess`` is now ready to run. It used the ``schema`` for the catalog to identify the error columns and it is now set up to procede. 

In [2]:
ARP()

Just like that, our ``AstrometricReductionProcess`` has been completed. You'll notice it has since appeared in the ``META`` table:

In [4]:
cmd.meta

Let's go ahead and take a look at the results of the reduction.

In [7]:
reduction_results = cmd.query(
    "SELECT CATOBJ, CATRA, CATDEC, SEPARATION ,CATNMATCH, RA, DEC, ASTROMETRIC_REDUCTION_SCORE FROM NED_STD_MATCH"
)

In [24]:
import matplotlib.pyplot as plt
import numpy as np

plt.hist(
    reduction_results["ASTROMETRIC_REDUCTION_SCORE"],
    bins=np.geomspace(1e-10, 1e-1, 20),
    ec="k",
    fc="darkgreen",
    alpha=0.35,
    density=True,
)
plt.hist(
    [
        np.amax(
            reduction_results.loc[
                reduction_results["CATOBJ"] == j, "ASTROMETRIC_REDUCTION_SCORE"
            ]
        )
        for j in reduction_results["CATOBJ"]
    ],
    bins=np.geomspace(1e-10, 1e-1, 20),
    ec="k",
    density=True,
    fc="red",
    alpha=0.35,
)
plt.xscale("log")